In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy
from numba import jit
import logging
from rich.logging import RichHandler

logging.basicConfig(
    level="INFO",
    format="%(message)s",
    datefmt="[%X]",
    handlers=[RichHandler(rich_tracebacks=True)]
)

from rich.traceback import install
install()


# всякие магиеческие метды
%load_ext autoreload
%autoreload 2

%matplotlib inline

%config InlineBackend.figure_format ='retina'

pd.set_option('display.max_rows', 30)
pd.options.display.max_columns = None
pd.options.display.float_format = '{:,.3f}'.format

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

<bound method InteractiveShell.excepthook of <ipykernel.zmqshell.ZMQInteractiveShell object at 0x111b67220>>

In [2]:
import settings
from customer_cases.eapteka.problem_formulation import *
from formats import api
from solvers.external.vrp_cli.solver import RustSolver

In [3]:
problem_params = AptekaParams(
    delay_pharmacy=5 * 60,
    delay_stock=10 * 60,
    pedestrian_max_weight=15,
    pedestrian_max_volume=40,
    driver_max_weight=15,
    driver_max_volume=40,
    point_delay=5,
)

In [4]:
data_folder = settings.DATA_DIR / 'eapteka/data'
points, stocks, couriers = load_data(
    coords_file=data_folder / "update_3.xlsx",
    orders_file=data_folder / "Заказы_13.xlsx",
    stocks_file=data_folder / "Склады.xlsx",
    couriers_file=data_folder / "Курьеры_13.xlsx",
)

In [5]:
stocks

,Наименование,Адрес,График работы,Широта,Долгота
0,"Мск, 1905 года 7","Москва г, 1905 года ул, дом № 7, строение 1 ...",09-21,55.762,37.559
1,"Мск, Бауманская 62-66","Москва г, Бауманская ул, дом № 62-66, строение...",09-21,55.766,37.679
2,"Мск, Волжский б-р 44","Москва г, Волжский б-р, дом № 44 ...",09-21,55.699,37.751
3,"Мск, Дмитрия Ульянова 5","Москва г, Дмитрия Ульянова ул, дом № 5 ...",09-21,55.692,37.562
4,"Мск, Дмитровское шоссе 87","Москва г, Дмитровское ш, дом № 87 ...",09-21,55.863,37.546
5,"Мск, Маломосковская 22/1","Москва г, Маломосковская ул, дом № 22, строени...",09-21,55.812,37.653
6,"Мск, Марьина роща склад","г. Москва, 17-й Проезд Марьиной рощи, 13с5",круглосуточно,55.808,37.605
7,"Мск, Мастеркова 3","Москва г, Мастеркова ул, дом № 3 ...",09-21,55.709,37.659
8,"Мск, Планерная 12к1","Москва г, Планерная ул, дом № 12, корпус 1",09-21,55.862,37.437
9,"Мск, Строгинский б-р 23","Москва г, Строгинский б-р, дом № 23 ...",09-21,55.803,37.407


In [6]:
# предобрабатываем датафреймы
stocks_df = preprocess_stocks(stocks)
couriers_df = preprocess_couriers(couriers)
points_df = preprocess_points(points)

In [7]:
fill_missing_columns(stocks=stocks_df, couriers=couriers_df, points=points_df, params=problem_params)

In [8]:
depots = build_depots(stocks=stocks_df)
depots

[Depot(id=0, name='1905 года 7', lat=55.7621, lon=37.55867, x=None, y=None, delay=300, time_windows=((9, 21),)),
 Depot(id=1, name='Бауманская 62-66', lat=55.766116, lon=37.678837, x=None, y=None, delay=300, time_windows=((9, 21),)),
 Depot(id=2, name='Волжский б-р  44', lat=55.699081, lon=37.750604, x=None, y=None, delay=300, time_windows=((9, 21),)),
 Depot(id=3, name='Дмитрия Ульянова 5', lat=55.692475, lon=37.561509, x=None, y=None, delay=300, time_windows=((9, 21),)),
 Depot(id=4, name='Дмитровское шоссе 87', lat=55.863015, lon=37.545761, x=None, y=None, delay=300, time_windows=((9, 21),)),
 Depot(id=5, name='Маломосковская 22/1', lat=55.81174, lon=37.652616, x=None, y=None, delay=300, time_windows=((9, 21),)),
 Depot(id=6, name='Марьина роща склад', lat=55.807506, lon=37.605337, x=None, y=None, delay=600, time_windows=((0, 24),)),
 Depot(id=7, name='Мастеркова 3', lat=55.708511, lon=37.658634, x=None, y=None, delay=300, time_windows=((9, 21),)),
 Depot(id=8, name='Планерная 12к1'

In [9]:
couriers_df

,id,name,profile,cost,priority,t_from,t_to,max_weight,max_volume
0,0,Акперов Ильгар Натиг Оглы,pedestrian,5,False,16,22,15.000,40.000
1,1,Бердников Евгений Леонидович,driver,10,False,9,21,15.000,40.000
2,2,Богомолов Даниил Денисович,driver,10,False,10,21,15.000,40.000
3,3,Бортников Сергей Сергеевич,pedestrian,5,True,9,14,15.000,40.000
4,4,Бортников Сергей Сергеевич,pedestrian,5,True,18,22,15.000,40.000
...,...,...,...,...,...,...,...,...,...
29,29,Салимов Рустам Мобинович,pedestrian,5,False,7,14,15.000,40.000
30,30,Салимов Рустам Мобинович,pedestrian,5,False,18,22,15.000,40.000
31,31,Тутубалин Андрей Валерьевич,pedestrian,5,False,10,21,15.000,40.000
32,32,Фетков Александр Геннадьевич,pedestrian,5,False,9,21,15.000,40.000


In [10]:
agents = build_agents(agents_df=couriers_df, depots=depots)

In [11]:
agents

[Agent(id=0, costs=AgentCosts(idle=0, departure=0, load=0, time=5, dist=0), amounts=[15.0, 40.0], time_windows=[16, 22], compatible_depots=[Depot(id=0, name='1905 года 7', lat=55.7621, lon=37.55867, x=None, y=None, delay=300, time_windows=((9, 21),)), Depot(id=1, name='Бауманская 62-66', lat=55.766116, lon=37.678837, x=None, y=None, delay=300, time_windows=((9, 21),)), Depot(id=2, name='Волжский б-р  44', lat=55.699081, lon=37.750604, x=None, y=None, delay=300, time_windows=((9, 21),)), Depot(id=3, name='Дмитрия Ульянова 5', lat=55.692475, lon=37.561509, x=None, y=None, delay=300, time_windows=((9, 21),)), Depot(id=4, name='Дмитровское шоссе 87', lat=55.863015, lon=37.545761, x=None, y=None, delay=300, time_windows=((9, 21),)), Depot(id=5, name='Маломосковская 22/1', lat=55.81174, lon=37.652616, x=None, y=None, delay=300, time_windows=((9, 21),)), Depot(id=6, name='Марьина роща склад', lat=55.807506, lon=37.605337, x=None, y=None, delay=600, time_windows=((0, 24),)), Depot(id=7, name='

In [12]:
jobs = build_jobs(points=points_df, depots=depots)
jobs

[Job(id=0, name='Москва, Королев г, Коминтерна улица, д.15', lat=55.92403, lon=37.81381, x=None, y=None, delay=5, time_windows=[(12, 14)], amounts=[0.0, 0.14], required_skills=frozenset(), price=4900.0, priority=False, depots=(Depot(id=6, name='Марьина роща склад', lat=55.807506, lon=37.605337, x=None, y=None, delay=600, time_windows=((0, 24),)),)),
 Job(id=1, name='Москва, Ангарская улица, 67к1', lat=55.8861, lon=37.52538, x=None, y=None, delay=5, time_windows=[(9, 18)], amounts=[0.972, 8.084], required_skills=frozenset(), price=1486.0, priority=False, depots=(Depot(id=6, name='Марьина роща склад', lat=55.807506, lon=37.605337, x=None, y=None, delay=600, time_windows=((0, 24),)),)),
 Job(id=2, name='г. Москва  ул Кировоградская  д.10', lat=55.626431, lon=37.606631, x=None, y=None, delay=5, time_windows=[(15, 21)], amounts=[0.0, 0.795], required_skills=frozenset(), price=2029.0, priority=False, depots=(Depot(id=6, name='Марьина роща склад', lat=55.807506, lon=37.605337, x=None, y=None,

In [18]:
from models.rich_vrp.geometries.geometry import HaversineGeometry
from functools import partial

problem_params = AptekaParams(
    delay_pharmacy=5 * 60,
    delay_stock=10 * 60,
    pedestrian_max_weight=15,
    pedestrian_max_volume=40,
    driver_max_weight=15,
    driver_max_volume=40,
    point_delay=5,
)

problem = build_eapteka_problem(
    settings.DATA_DIR / 'eapteka/data',
    params=problem_params,
    profile_geometries={
        'driver': partial(HaversineGeometry, default_speed=15),
        'pedestrian': partial(HaversineGeometry, default_speed=1.5),
    }
)

# solver = RustSolver()
# solution = solver.solve(problem)
# solution_str = api.export(solution)

TypeError: unhashable type: 'list'